In [2]:
import tensorflow as tf

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

2024-02-13 13:17:16.429191: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-13 13:17:16.429242: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-13 13:17:16.430754: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-13 13:17:16.438865: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-13 13:17:17.466886: W tensorflow/compiler/tf2

In [10]:
# 3
train_image_generator = ImageDataGenerator(rescale=1. / 255,
                                             rotation_range=90,
                                             width_shift_range=0.2,
                                             height_shift_range=0.2,
                                             shear_range=0.2,
                                             zoom_range=0.2,
                                             horizontal_flip=True,
                                             fill_mode='nearest')
validation_image_generator = ImageDataGenerator(rescale=1. / 255,
                                             rotation_range=90,
                                             width_shift_range=0.2,
                                             height_shift_range=0.2,
                                             shear_range=0.2,
                                             zoom_range=0.2,
                                             horizontal_flip=True,
                                             fill_mode='nearest')
# test_image_generator = ImageDataGenerator(rescale=1. / 255)

train_data_gen = train_image_generator.flow_from_directory(batch_size=128,
      directory='cats_and_dogs/train',
      shuffle=True,
      target_size=(150, 150),
      class_mode='binary')

val_data_gen = validation_image_generator.flow_from_directory(batch_size=128,
        directory='cats_and_dogs/validation',
        target_size=(150, 150),
        class_mode='binary')

# test_data_gen = test_image_generator.flow_from_directory(
#     directory=test_dir,
#     target_size=(IMG_HEIGHT, IMG_WIDTH),
#     class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [11]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', 
           input_shape=(150, 150 ,3)),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 150, 150, 16)      448       
                                                                 
 max_pooling2d_6 (MaxPoolin  (None, 75, 75, 16)        0         
 g2D)                                                            
                                                                 
 dropout_2 (Dropout)         (None, 75, 75, 16)        0         
                                                                 
 conv2d_7 (Conv2D)           (None, 75, 75, 32)        4640      
                                                                 
 max_pooling2d_7 (MaxPoolin  (None, 37, 37, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_8 (Conv2D)           (None, 37, 37, 64)       

In [38]:
history = model.fit(
    train_data_gen,
    steps_per_epoch=2000 // 128,
    epochs=200,
    validation_data=val_data_gen,
    validation_steps=1000 // 128
)

Epoch 1/200
15/15 [==============================] - 28s 2s/step - loss: 0.5170 - accuracy: 0.7302 - val_loss: 0.5816 - val_accuracy: 0.7031
Epoch 2/200
15/15 [==============================] - 27s 2s/step - loss: 0.5047 - accuracy: 0.7505 - val_loss: 0.5565 - val_accuracy: 0.7310
Epoch 3/200
15/15 [==============================] - 31s 2s/step - loss: 0.5015 - accuracy: 0.7521 - val_loss: 0.5849 - val_accuracy: 0.7098
Epoch 4/200
15/15 [==============================] - 34s 2s/step - loss: 0.4914 - accuracy: 0.7660 - val_loss: 0.5416 - val_accuracy: 0.7355
Epoch 5/200
15/15 [==============================] - 35s 2s/step - loss: 0.5051 - accuracy: 0.7468 - val_loss: 0.5724 - val_accuracy: 0.7087
Epoch 6/200
15/15 [==============================] - 36s 2s/step - loss: 0.5029 - accuracy: 0.7557 - val_loss: 0.5526 - val_accuracy: 0.7076
Epoch 7/200
15/15 [==============================] - 35s 2s/step - loss: 0.5012 - accuracy: 0.7511 - val_loss: 0.6233 - val_accuracy: 0.6674
Epoch 8/200
1

In [41]:
import pandas as pd
import re
# Define the test data generator without augmentation
test_image_generator = ImageDataGenerator(rescale=1. / 255)

# Provide the path to the directory containing your test images
test_dir = 'cats_and_dogs/test/'

file_names = [f for f in os.listdir(test_dir) if f.endswith('.jpg')]

# Sort the JPEG files based on their numbers
file_names_sorted = sorted(file_names, key=lambda x: int(re.findall(r'\d+', x)[0]))

print(file_names_sorted)

file_paths = [os.path.join(test_dir, file_name) for file_name in file_names_sorted]
df_test = pd.DataFrame({'file_paths': file_paths})
print(df_test)

# Generate the test data from the directory
test_data_gen = test_image_generator.flow_from_dataframe(
    dataframe=df_test,
    x_col='file_paths',
    target_size=(150, 150),  # Specify the target size of the images
    batch_size=128,
    class_mode=None,  # Adjust class_mode based on your task (e.g., 'categorical' for multi-class classification)
    shuffle=False  # Set shuffle to False to preserve the order of predictions
)


['1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '20.jpg', '21.jpg', '22.jpg', '23.jpg', '24.jpg', '25.jpg', '26.jpg', '27.jpg', '28.jpg', '29.jpg', '30.jpg', '31.jpg', '32.jpg', '33.jpg', '34.jpg', '35.jpg', '36.jpg', '37.jpg', '38.jpg', '39.jpg', '40.jpg', '41.jpg', '42.jpg', '43.jpg', '44.jpg', '45.jpg', '46.jpg', '47.jpg', '48.jpg', '49.jpg', '50.jpg']
                   file_paths
0    cats_and_dogs/test/1.jpg
1    cats_and_dogs/test/2.jpg
2    cats_and_dogs/test/3.jpg
3    cats_and_dogs/test/4.jpg
4    cats_and_dogs/test/5.jpg
5    cats_and_dogs/test/6.jpg
6    cats_and_dogs/test/7.jpg
7    cats_and_dogs/test/8.jpg
8    cats_and_dogs/test/9.jpg
9   cats_and_dogs/test/10.jpg
10  cats_and_dogs/test/11.jpg
11  cats_and_dogs/test/12.jpg
12  cats_and_dogs/test/13.jpg
13  cats_and_dogs/test/14.jpg
14  cats_and_dogs/test/15.jpg
15  cats_and_dogs/test/16.jp

In [42]:

probabilities = model.predict(test_data_gen)
print(probabilities)

probabilities = np.round(probabilities).flatten()
print(probabilities)
answers =  [1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0,
            1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0,
            1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1,
            1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1,
            0, 0, 0, 0, 0, 0]

correct = 0

for probability, answer in zip(probabilities, answers):
  if round(probability) == answer:
    correct +=1

percentage_identified = (correct / len(answers)) * 100

passed_challenge = percentage_identified >= 63

print(f"Your model correctly identified {round(percentage_identified, 2)}% of the images of cats and dogs.")

if passed_challenge:
  print("You passed the challenge!")
else:
  print("You haven't passed yet. Your model should identify at least 63% of the images. Keep trying. You will get it!")

1/1 [==============================] - 0s 214ms/step
[[9.81465220e-01]
 [9.98782635e-01]
 [1.51115984e-01]
 [1.00000000e+00]
 [6.27085046e-06]
 [3.79781038e-01]
 [4.74147128e-05]
 [2.97435485e-02]
 [1.78967893e-01]
 [1.40645809e-03]
 [6.57230703e-06]
 [9.99999642e-01]
 [1.81722280e-04]
 [5.86648464e-01]
 [2.23535649e-03]
 [1.52285015e-02]
 [9.97877538e-01]
 [9.48721409e-01]
 [3.26444581e-03]
 [1.51607525e-02]
 [6.89003527e-01]
 [2.96397448e-01]
 [8.61658573e-01]
 [9.99999821e-01]
 [1.01447418e-01]
 [9.40146208e-01]
 [8.46473694e-01]
 [3.36525991e-04]
 [2.72745639e-01]
 [9.91060615e-01]
 [9.61006880e-01]
 [2.71565001e-02]
 [1.00000000e+00]
 [3.59066218e-01]
 [2.75982600e-02]
 [4.54528302e-01]
 [1.46636348e-02]
 [9.15426881e-07]
 [9.90282774e-01]
 [6.01697862e-02]
 [4.16343361e-02]
 [9.93492603e-01]
 [9.98973548e-01]
 [8.09547782e-01]
 [5.34176119e-02]
 [2.25218534e-01]
 [1.27573294e-04]
 [3.12010944e-01]
 [9.79464054e-01]
 [3.54393455e-03]]
[1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 